## Imports

In [ ]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm, trange
import torch
from transformers import ViTFeatureExtractor, ViTForImageClassification
from datasets import load_dataset
from sklearn.linear_model import LinearRegression
import seaborn as sns

tqdm.pandas()
sns.set_context('talk')
sns.set_style('whitegrid')

## Functions

In [ ]:
def extract_hidden_states(image):
    inputs = feature_extractor(images=image, return_tensors='pt')
    outputs = model(**inputs, output_hidden_states=True)
    batch_index = 0
    cls_index = 0
    return [hidden_state[batch_index, cls_index, :]
            for hidden_state in outputs.hidden_states]


def predict(hidden_state):
    logits = model.classifier(hidden_state).detach().numpy()
    pred = logits.argmax() + 1
    p = np.exp(logits.max()) / np.sum(np.exp(logits))
    return {
        'pred': pred,
        'p': p
    }


def reverse_dict(dictionary):
    reversed_dict = {value: key for key, value in dictionary.items()}
    return reversed_dict


def get_early_exit_layer_and_pred(preds, threshold):
    for layer_index in range(len(preds) - threshold + 1):
        if len(set(preds[layer_index: layer_index + threshold])) == 1:
            return {
                'early_exit_layer':  layer_index + threshold - 1,
                'early_exit_pred':  preds[layer_index]
            }
    return {
        'early_exit_layer':  len(preds) - 1,
        'early_exit_pred':  preds[-1]
    }


def get_early_exit(df, pred_col):
    res = []
    for threshold in trange(1, df['layer_index'].max() + 1):
        df_early_exit = df.groupby(['input_index', 'label'])[pred_col].apply(list).reset_index()

        df_early_exit['early_exit_res'] = df_early_exit[pred_col].apply(
            get_early_exit_layer_and_pred,
            threshold=threshold
        )

        df_early_exit = pd.concat([
            df_early_exit,
            pd.json_normalize(df_early_exit['early_exit_res'])
        ], axis=1)
        df_early_exit['accuracy'] = df_early_exit['label'] == df_early_exit['early_exit_pred']

        res.append(df_early_exit[['accuracy', 'early_exit_layer']].mean())

    return pd.DataFrame(res)

## Download the huggingface validation set and ViT

In [ ]:
feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-large-patch16-224')
model = ViTForImageClassification.from_pretrained('google/vit-large-patch16-224')
dataset = load_dataset('imagenet-1k', streaming=True)

## Create a DataFrame holding all predictions according to different hidden state layers

In [ ]:
res = []

# We can't use the entire dataset without running out of memory.
# It would require more complicated data handling to solve these issues.
# Complicating things is against the One Shot Learning philosophy,
# so we randomly select a subset of the data.
random_labels = set(np.random.RandomState(42).choice(list(model.config.id2label.keys()), 200))
res = []

for input_index, example in tqdm(enumerate(dataset['validation']), total=50000):
    if input_index > 0 and input_index % 1000 == 0:
        pd.DataFrame(res).to_json(f'data/{input_index}.jsonl', lines=True, orient='records')
        res = []
        
    if example['label'] not in random_labels or example['image'].mode != 'RGB':
        continue
    res += [{
        'input_index': input_index,
        'label': example['label'],
        'layer_index': layer_index,
        'hidden_state': hidden_state.detach().numpy(),
        **predict(hidden_state)
    } for layer_index, hidden_state in enumerate(extract_hidden_states(example['image']))]
        
if len(res) > 0:
    pd.DataFrame(res).to_json(f'data/{input_index + 1}.jsonl', lines=True, orient='records')

In [ ]:
df = pd.concat([pd.read_json(f'data/{filename}', lines=True)
                for filename in tqdm(os.listdir('data'))])

In [ ]:
early_exit = get_early_exit(df, 'pred')

## plot early exit prediction

In [ ]:
sns.lineplot(x= early_exit['early_exit_layer'],
             y=early_exit['accuracy'],
             marker='o')

In [ ]:
sns.lineplot(x=df['layer_index'],
             y=df['pred'] == df['label'])

In [ ]:
sns.lineplot(x=df['layer_index'],
             y=df['p'])

## plot the Distace between different hidden state layers

In [ ]:
df['last_hidden_state'] = df.groupby('input_index')['hidden_state'].transform('last')
df['hidden_state_dist'] = df.progress_apply(
    lambda row: np.linalg.norm(np.array(row['hidden_state']) -
                               np.array(row['last_hidden_state'])),
    axis=1
)

In [ ]:
sns.lineplot(x=df['layer_index'],
             y=df['hidden_state_dist'])

# Create the linear regression for making different layers "closer" 

In [ ]:
df_train = df.iloc[: int(len(df) * 0.8)]
df_test = df.iloc[int(len(df) * 0.8):]

projectors = []
for layer_index in trange(df_train['layer_index'].max() + 1):
    df_to_use = df_train[df_train['layer_index'] == layer_index]
    X = np.stack(df_to_use['hidden_state'])
    y = np.stack(df_to_use['last_hidden_state'])
    projectors.append(LinearRegression().fit(X, y))

In [ ]:
df_test['proj_hidden_state'] = df_test.progress_apply(
    lambda row: projectors[row['layer_index']].predict([row['hidden_state']])[0],
    axis=1
)

In [ ]:
df_test['proj_pred'] = df_test['proj_hidden_state'].progress_apply(
    lambda proj_hidden_state: predict(torch.Tensor(proj_hidden_state))['pred']
)

In [ ]:
early_exit_before = get_early_exit(df_test, 'pred')
early_exit_proj = get_early_exit(df_test, 'proj_pred')

## Plot the effect of the projections

In [ ]:
sns.lineplot(x= early_exit_before['early_exit_layer'],
             y=early_exit_before['accuracy'],
             marker='o',
             label='before')

sns.lineplot(x= early_exit_proj['early_exit_layer'],
             y=early_exit_proj['accuracy'],
             marker='o',
             label='projected')